# Language Agent Tree Search

[LATS (Language Agent Tree Search)](https://arxiv.org/pdf/2310.04406.pdf) by Zhou et al. combines LLM capabilities in planning, acting, and reasoning within a Monte Carlo tree search framework, allowing for deliberate and adaptive problem-solving guided by external feedback and self-reflection.

We've implemented this agent as a LlamaPack - you can either pip install it to run it out-of-the-box or call `download_llama_pack` to load the pack.

## Setup

#### Option 1. pip install llama-index-packs-lats

In [ ]:
!pip install llama-index-packs-lats

#### Option 2. `download_llama_pack`

In [ ]:
# from llama_index.core.llama_pack import download_llama_pack

# LATSPacks = download_llama_pack("LATSPack")

### Define Global Settings

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings


llm = OpenAI(model="gpt-3.5-turbo")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.storage import StorageContext
import os


if not os.path.exists("./storage/lyft"):
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
else:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/uber")
    uber_index = load_index_from_storage(storage_context)

### Setup Tools

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup Agent

Now we can setup the LATS agent.

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.packs.lats import LATSAgentWorker


agent_worker = LATSAgentWorker.from_tools(query_engine_tools, llm=llm, verbose=True)
agent = AgentRunner(agent_worker)

## Run Some Queries

In [ ]:
response = agent.query(
    "Given the revenue growth and risk factors of Uber and Lyft, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)
print(str(response))

> Selecting node to expand: Observation: Given the revenue growth and risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.
> Generated new reasoning step: Thought: I need to compare the revenue growth and risk factors of Uber and Lyft to determine which company is performing better.
Action: uber_10k
Action Input: {'input': 'revenue growth and risk factors'}
Observation: The text provided discusses the application of funds and the appointment of the Administrative Agent in a financial agreement. It also touches on waivers, amendments, and the role of sub-agents in the agreement.
> Generated new reasoning step: Thought: I need to compare the revenue growth and risk factors of Uber and Lyft to determine which company is performing better.
Action: lyft_10k
Action Input: {'input': 'revenue growth and risk factors'}
Observation: The risk factors mentioned in the context include potential issues with third-party payment proces

ValueError: "SearchNode" object has no field "score"